# Introducción a MLFlow y Databricks: acelerando el Machine Learning LifeCycle - Python Sevilla 2019

## MLFlow Tracking

### Basic example

In [1]:
import mlflow

In [2]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment('test_1')

In [3]:
run =  mlflow.start_run()
# with mlflow.start_run() as run: -> another alternative

In [4]:
mlflow.log_param('param1', 1)
mlflow.log_metric('metric1', 2)

In [5]:
mlflow.log_param('param1', 1)
mlflow.log_metric('metric1', 2)

In [6]:
mlflow.end_run()

### Breast cancer: Scikit-learn

In [7]:
mlflow.set_experiment('breast_cancer')

In [8]:
import numpy as np
import pandas
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [9]:
cancer = load_breast_cancer()
cancer.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [10]:
X = np.array(cancer.data)
y = np.array(cancer.target)
print(f'X: {X.shape}, y: {y.shape}')

X: (569, 30), y: (569,)


In [11]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=426, test_size=143, random_state=0)

In [12]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [17]:
# Model 1: Logistic Regression
def breast_cancer_lr(solver="lbfgs", C=1.0):
    from sklearn.linear_model import LogisticRegression
    import mlflow.sklearn
    with mlflow.start_run() as run:
        lr = LogisticRegression(solver = solver, C = C)
        mlflow.log_param("solver", solver)
        mlflow.log_param("C", C)
        mlflow.set_tag("model type", "sklearn - LogisticRegression")
        lr.fit(x_train, y_train)
        accuracy = lr.score(x_test, y_test)
        print("Accuracy: %s" % accuracy)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.sklearn.log_model(lr, "model")
        print("Model saved in run %s" % mlflow.active_run().info.run_uuid)

In [18]:
breast_cancer_lr()
breast_cancer_lr(solver="liblinear")
breast_cancer_lr(solver="liblinear", C=0.5)

Accuracy: 0.958041958041958
Model saved in run b191ff233bb842468dbf1f46fcac5de7
Accuracy: 0.958041958041958
Model saved in run c67d75d18f9640f7a673fa1a5f2ab4a5
Accuracy: 0.972027972027972
Model saved in run ef593013b297488aa90b547cd400b617


In [19]:
# Model 2: Random Forest
def breast_cancer_rf(n_estimators=100, max_depth=2, criterion="gini"):
    from sklearn.ensemble import RandomForestClassifier
    import mlflow.sklearn
    with mlflow.start_run() as run:
        clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, criterion=criterion)
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("criterion", criterion)
        mlflow.set_tag("model type", "sklearn - RandomForest")
        clf.fit(x_train, y_train)
        accuracy = clf.score(x_test, y_test)
        print("Accuracy: %s" % accuracy)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.sklearn.log_model(clf, "model")
        print("Model saved in run %s" % mlflow.active_run().info.run_uuid)

In [20]:
breast_cancer_rf()
breast_cancer_rf(max_depth=5)
breast_cancer_rf(n_estimators=500, criterion="entropy")

Accuracy: 0.965034965034965
Model saved in run b8fe5a2fdcc94cb8986c65413706ebfe
Accuracy: 0.965034965034965
Model saved in run 2d1e13a4e7af4a8ca79e97e6d7459434
Accuracy: 0.958041958041958
Model saved in run 44a805bc3816468291d086844cb4c428


In [53]:
# Model 3: Keras
from keras.callbacks import Callback
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_epoch_end(self, epoch, logs={}):
        loss = logs.get('loss')
        acc = logs.get('accuracy')
        mlflow.log_metric("loss", loss, step=epoch)
        mlflow.log_metric("val_accuracy", acc, step=epoch)
        precision, recall, accuracy = validate_keras(self.model, x_test, y_test)        
        mlflow.log_metric("precision", precision, step=epoch)
        mlflow.log_metric("recall", recall, step=epoch)
        mlflow.log_metric("accuracy", accuracy, step=epoch)
        self.losses.append(loss)
        
        
def validate_keras(model, x_test, y_test):    
    y_pred = model.predict(x_test)
    y_pred = (y_pred > 0.5)
    from sklearn.metrics import confusion_matrix
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    return precision, recall, accuracy
    

def breast_cancer_keras(optimizer='adam',dropout=0.00, nb_epoch=20):
    import mlflow.keras
    import keras
    from keras.models import Sequential
    from keras.layers import Dense, Dropout
    with mlflow.start_run() as run:
        mlflow.set_tag("model type", "keras - MLP")
        model = Sequential()
        # Adding the input layer and the first hidden layer
        model.add(Dense(output_dim=16, init='uniform', activation='relu', input_dim=30))
        # Adding dropout to prevent overfitting
        model.add(Dropout(p=dropout))
        # Adding the second hidden layer
        model.add(Dense(output_dim=16, init='uniform', activation='relu'))
        # Adding dropout to prevent overfitting
        model.add(Dropout(p=dropout))
        # Adding the output layer
        model.add(Dense(output_dim=1, init='uniform', activation='sigmoid'))
        # Compiling the ANN
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        history = LossHistory()
        model.fit(x_train, y_train, batch_size=100, nb_epoch=nb_epoch, callbacks=[history])
        precision, recall, accuracy = validate_keras(model, x_test, y_test)
        mlflow.log_param("optimizer", optimizer)
        mlflow.log_param("dropout", dropout)
        mlflow.keras.log_model(model, "model")

In [50]:
breast_cancer_keras()

Epoch 1/20
426/426 [==============================] - 0s 796us/step - loss: 0.6928 - accuracy: 0.6268
Epoch 2/20
426/426 [==============================] - 0s 68us/step - loss: 0.6912 - accuracy: 0.6268
Epoch 3/20
426/426 [==============================] - 0s 68us/step - loss: 0.6887 - accuracy: 0.6268
Epoch 4/20
426/426 [==============================] - 0s 54us/step - loss: 0.6848 - accuracy: 0.6268
Epoch 5/20
426/426 [==============================] - 0s 68us/step - loss: 0.6781 - accuracy: 0.6315
Epoch 6/20
426/426 [==============================] - 0s 44us/step - loss: 0.6687 - accuracy: 0.6408
Epoch 7/20
426/426 [==============================] - 0s 64us/step - loss: 0.6550 - accuracy: 0.6690
Epoch 8/20
426/426 [==============================] - 0s 52us/step - loss: 0.6366 - accuracy: 0.7277
Epoch 9/20
426/426 [==============================] - 0s 77us/step - loss: 0.6104 - accuracy: 0.7911
Epoch 10/20
426/426 [==============================] - 0s 51us/step - loss: 0.5807 - accur

In [54]:
breast_cancer_keras(nb_epoch=100)

Epoch 1/100
426/426 [==============================] - 0s 526us/step - loss: 0.6927 - accuracy: 0.6127
Epoch 2/100
426/426 [==============================] - 0s 61us/step - loss: 0.6909 - accuracy: 0.6690
Epoch 3/100
426/426 [==============================] - 0s 62us/step - loss: 0.6884 - accuracy: 0.7394
Epoch 4/100
426/426 [==============================] - 0s 31us/step - loss: 0.6845 - accuracy: 0.8380
Epoch 5/100
426/426 [==============================] - 0s 31us/step - loss: 0.6787 - accuracy: 0.9038
Epoch 6/100
426/426 [==============================] - 0s 32us/step - loss: 0.6701 - accuracy: 0.9249
Epoch 7/100
426/426 [==============================] - 0s 26us/step - loss: 0.6582 - accuracy: 0.9413
Epoch 8/100
426/426 [==============================] - 0s 34us/step - loss: 0.6416 - accuracy: 0.9413
Epoch 9/100
426/426 [==============================] - 0s 33us/step - loss: 0.6201 - accuracy: 0.9484
Epoch 10/100
426/426 [==============================] - 0s 35us/step - loss: 0.59

426/426 [==============================] - 0s 44us/step - loss: 0.0504 - accuracy: 0.9883
Epoch 81/100
426/426 [==============================] - 0s 36us/step - loss: 0.0501 - accuracy: 0.9883
Epoch 82/100
426/426 [==============================] - 0s 42us/step - loss: 0.0498 - accuracy: 0.9883
Epoch 83/100
426/426 [==============================] - 0s 34us/step - loss: 0.0495 - accuracy: 0.9883
Epoch 84/100
426/426 [==============================] - 0s 44us/step - loss: 0.0492 - accuracy: 0.9883
Epoch 85/100
426/426 [==============================] - 0s 38us/step - loss: 0.0489 - accuracy: 0.9883
Epoch 86/100
426/426 [==============================] - 0s 38us/step - loss: 0.0486 - accuracy: 0.9883
Epoch 87/100
426/426 [==============================] - 0s 45us/step - loss: 0.0484 - accuracy: 0.9883
Epoch 88/100
426/426 [==============================] - 0s 38us/step - loss: 0.0481 - accuracy: 0.9883
Epoch 89/100
426/426 [==============================] - 0s 41us/step - loss: 0.0478 - 

In [55]:
breast_cancer_keras(dropout=0.25, nb_epoch=100)

Epoch 1/100
426/426 [==============================] - 0s 794us/step - loss: 0.6924 - accuracy: 0.7019
Epoch 2/100
426/426 [==============================] - 0s 72us/step - loss: 0.6903 - accuracy: 0.7840
Epoch 3/100
426/426 [==============================] - 0s 71us/step - loss: 0.6871 - accuracy: 0.8169
Epoch 4/100
426/426 [==============================] - 0s 70us/step - loss: 0.6825 - accuracy: 0.8709
Epoch 5/100
426/426 [==============================] - 0s 44us/step - loss: 0.6751 - accuracy: 0.8991
Epoch 6/100
426/426 [==============================] - 0s 42us/step - loss: 0.6651 - accuracy: 0.9178
Epoch 7/100
426/426 [==============================] - 0s 42us/step - loss: 0.6495 - accuracy: 0.9460
Epoch 8/100
426/426 [==============================] - 0s 41us/step - loss: 0.6318 - accuracy: 0.9460
Epoch 9/100
426/426 [==============================] - 0s 72us/step - loss: 0.6053 - accuracy: 0.9507
Epoch 10/100
426/426 [==============================] - 0s 52us/step - loss: 0.57

426/426 [==============================] - 0s 39us/step - loss: 0.0577 - accuracy: 0.9883
Epoch 81/100
426/426 [==============================] - 0s 66us/step - loss: 0.0651 - accuracy: 0.9883
Epoch 82/100
426/426 [==============================] - 0s 48us/step - loss: 0.0523 - accuracy: 0.9883
Epoch 83/100
426/426 [==============================] - 0s 34us/step - loss: 0.0552 - accuracy: 0.9859
Epoch 84/100
426/426 [==============================] - 0s 48us/step - loss: 0.0574 - accuracy: 0.9883
Epoch 85/100
426/426 [==============================] - 0s 41us/step - loss: 0.0575 - accuracy: 0.9859
Epoch 86/100
426/426 [==============================] - 0s 42us/step - loss: 0.0645 - accuracy: 0.9859
Epoch 87/100
426/426 [==============================] - 0s 50us/step - loss: 0.0536 - accuracy: 0.9883
Epoch 88/100
426/426 [==============================] - 0s 56us/step - loss: 0.0599 - accuracy: 0.9883
Epoch 89/100
426/426 [==============================] - 0s 38us/step - loss: 0.0522 - 